# Phase 3 — Blocking & Candidate Generation

**Input:** Cleaned parquet files from Phase 2 (`../artifacts/phase2_preprocessing/`)  
**Output:** Deduplicated candidate pairs + blocking summary (`../artifacts/phase3_blocking/`)

### Approach
1. **Information-theoretic analysis** — pick the best blocking keys using entropy & mutual information
2. **Traditional blocking** — Strategies A, B, C (increasing specificity)
3. **Advanced blocking** — Canopy clustering (TF-IDF + cosine) and LSH (MinHash)
4. **Union + evaluation** — combine all strategies, measure quality, export

## 3.1 — Load Cleaned Datasets

In [1]:
import os, gc
import pandas as pd
import numpy as np

print('3.1 — LOAD CLEANED DATASETS')
print('-' * 60)

INPUT_DIR  = '../artifacts/phase2_preprocessing'
OUTPUT_DIR = '../artifacts/phase3_blocking'
os.makedirs(OUTPUT_DIR, exist_ok=True)

op_clean    = pd.read_parquet(os.path.join(INPUT_DIR, 'open_payments_clean.parquet'))
med_clean   = pd.read_parquet(os.path.join(INPUT_DIR, 'medicare_clean.parquet'))
pecos_clean = pd.read_parquet(os.path.join(INPUT_DIR, 'pecos_clean.parquet'))

op_tier2 = op_clean[op_clean['linkage_tier'] == 'tier2_fuzzy'].copy().reset_index(drop=True)
med_clean = med_clean.reset_index(drop=True)

full_cross = len(op_tier2) * len(med_clean)

print(f'OP tier2_fuzzy records:  {len(op_tier2):>12,}')
print(f'Medicare records:        {len(med_clean):>12,}')
print(f'PECOS records:           {len(pecos_clean):>12,}')
print(f'Full cross-product:      {full_cross:>12,} pairs')

del op_clean; gc.collect()
print('\n✓ Datasets loaded.')

3.1 — LOAD CLEANED DATASETS
------------------------------------------------------------
OP tier2_fuzzy records:         4,683
Medicare records:           1,175,281
PECOS records:              2,936,748
Full cross-product:      5,503,840,923 pairs

✓ Datasets loaded.


## 3.2 — Blocking Key Selection via Information Theory

Before picking blocking keys, we need a **data-driven way** to decide which
attributes to combine. Two simple ideas from information theory help:

- **Entropy** — How spread out are the values? An attribute with many distinct
  values (high entropy) creates smaller, tighter blocks. State has ~53 values
  (low entropy), while Last Name Soundex has thousands (high entropy).

- **Mutual Information** — How much do two attributes overlap? If State and
  Soundex tell you very *different* things about a record (low MI), combining
  them gives you a much better blocking key. If they're redundant (high MI),
  combining them doesn't help much.

We compute these for every candidate attribute, then pick the combination
with the **highest joint entropy** (most distinct blocks) and **lowest MI**
(least redundancy).

In [2]:
print('3.2 — BLOCKING KEY SELECTION VIA INFORMATION THEORY')
print('-' * 60)

def shannon_entropy(series):
    """Entropy in bits — higher means more distinct values."""
    counts = series.dropna().value_counts()
    probs = counts / counts.sum()
    return -(probs * np.log2(probs)).sum()

def joint_entropy(s1, s2):
    """Joint entropy of two attributes combined."""
    mask = s1.notna() & s2.notna()
    combined = s1[mask].astype(str) + '||' + s2[mask].astype(str)
    return shannon_entropy(combined)

def mutual_information(s1, s2):
    """MI = how much knowing one tells you about the other."""
    mask = s1.notna() & s2.notna()
    return shannon_entropy(s1[mask]) + shannon_entropy(s2[mask]) - joint_entropy(s1, s2)

# Candidate blocking attributes (OP side)
op_attrs = {
    'State':      op_tier2['Recipient_State'],
    'LN Soundex': op_tier2['LAST_NAME_SOUNDEX'],
    'FN Soundex': op_tier2['FIRST_NAME_SOUNDEX'],
    'Last Name':  op_tier2['Covered_Recipient_Last_Name'].str.upper(),
    'First Name': op_tier2['Covered_Recipient_First_Name'].str.upper(),
    'ZIP5':       op_tier2['Recipient_Zip5'],
}

# 1) Single-attribute entropy
print('\n--- Single-Attribute Entropy (bits) ---')
print(f"{'Attribute':<15} {'Entropy':>8} {'Unique Vals':>12}")
print('-' * 37)
for name, series in op_attrs.items():
    print(f'{name:<15} {shannon_entropy(series):>8.3f} {series.dropna().nunique():>12,}')

# 2) Pairwise MI
print('\n--- Pairwise Mutual Information (bits) ---')
print('(Lower = more independent = better to combine)')
names = list(op_attrs.keys())
header = f"{'':>15}"
for a in names: header += f' {a[:9]:>9}'
print(header)
for a in names:
    row = f'{a:<15}'
    for b in names:
        if a == b: row += f" {'—':>9}"
        else: row += f' {mutual_information(op_attrs[a], op_attrs[b]):>9.3f}'
    print(row)

# 3) Composite key joint entropy
print('\n--- Composite Key Joint Entropy ---')
print('(Higher = more distinct blocks = better reduction)')
combos = [
    ('State', 'LN Soundex'),
    ('State', 'Last Name'),
    ('State', 'FN Soundex', 'LN Soundex'),
    ('State', 'Last Name', 'First Name'),
    ('State', 'ZIP5'),
]
print(f"{'Key Combination':<45} {'Entropy':>8} {'Blocks':>10}")
print('-' * 65)
for combo in combos:
    parts = [op_attrs[c].fillna('').astype(str) for c in combo]
    combined = parts[0]
    for p in parts[1:]: combined = combined + '||' + p
    print(f"{' + '.join(combo):<45} {shannon_entropy(combined):>8.3f} {combined.nunique():>10,}")

# 4) Independence check
print('\n--- Independence Check ---')
print('Ratio close to 1.0 = independent (good to combine)')
for a, b in [('State','LN Soundex'),('FN Soundex','LN Soundex'),('State','FN Soundex')]:
    h_a, h_b = shannon_entropy(op_attrs[a]), shannon_entropy(op_attrs[b])
    h_ab = joint_entropy(op_attrs[a], op_attrs[b])
    ratio = h_ab / (h_a + h_b) if (h_a + h_b) > 0 else 0
    print(f'  {a} + {b}: independence ratio = {ratio:.4f}')

print('\n✓ Analysis complete.')
print('  → State + LN Soundex: high joint entropy, low MI → best 2-key combo (Strategy A)')
print('  → Adding FN Soundex: further tightens blocks with minimal redundancy (Strategy C)')

3.2 — BLOCKING KEY SELECTION VIA INFORMATION THEORY
------------------------------------------------------------

--- Single-Attribute Entropy (bits) ---
Attribute        Entropy  Unique Vals
-------------------------------------
State              4.586           53
LN Soundex         9.868        1,611
FN Soundex         8.282          764
Last Name         11.319        3,385
First Name         9.746        1,757
ZIP5              10.715        2,540

--- Pairwise Mutual Information (bits) ---
(Lower = more independent = better to combine)
                    State LN Sounde FN Sounde Last Name First Nam      ZIP5
State                   —     2.680     1.610     3.858     2.594     4.572
LN Soundex          2.680         —     6.017     9.868     7.451     8.414
FN Soundex          1.610     6.017         —     7.434     8.282     6.845
Last Name           3.858     9.868     7.434         —     8.887     9.845
First Name          2.594     7.451     8.282     8.887         —     8

## 3.3 — Strategy A: State + Last Name Soundex

Justified by 3.2: State and LN Soundex are near-independent with high joint
entropy — the best 2-attribute blocking key.

In [3]:
print('3.3 — STRATEGY A: STATE + LAST NAME SOUNDEX')
print('-' * 60)

op_tier2['_block_A'] = op_tier2['LAST_NAME_SOUNDEX'].fillna('') + '_' + op_tier2['Recipient_State'].fillna('')
med_clean['_block_A'] = med_clean['LAST_NAME_SOUNDEX'].fillna('') + '_' + med_clean['Rndrng_Prvdr_State_Abrvtn'].fillna('')

pairs_A = (
    op_tier2[['_block_A']].reset_index().rename(columns={'index':'index_op'})
    .merge(med_clean[['_block_A']].reset_index().rename(columns={'index':'index_med'}), on='_block_A')
    [['index_op','index_med']]
)

rr_A = 1 - len(pairs_A) / full_cross
print(f'Candidate pairs (A):  {len(pairs_A):>12,}')
print(f'Reduction ratio:      {rr_A*100:.6f}%')
op_tier2.drop(columns='_block_A', inplace=True)
med_clean.drop(columns='_block_A', inplace=True)
gc.collect()
print('✓ Strategy A complete.')

3.3 — STRATEGY A: STATE + LAST NAME SOUNDEX
------------------------------------------------------------
Candidate pairs (A):       427,752
Reduction ratio:      99.992228%
✓ Strategy A complete.


## 3.4 — Strategy B: Exact Last Name (upper) + State

Stricter than A — uses exact uppercased last name instead of Soundex.
More unique values (higher entropy), avoids phonetic collisions.

In [4]:
print('3.4 — STRATEGY B: EXACT LAST NAME + STATE')
print('-' * 60)

op_tier2['_block_B'] = op_tier2['Covered_Recipient_Last_Name'].fillna('').str.upper() + '_' + op_tier2['Recipient_State'].fillna('')
med_clean['_block_B'] = med_clean['Rndrng_Prvdr_Last_Org_Name'].fillna('').str.upper() + '_' + med_clean['Rndrng_Prvdr_State_Abrvtn'].fillna('')

pairs_B = (
    op_tier2[['_block_B']].reset_index().rename(columns={'index':'index_op'})
    .merge(med_clean[['_block_B']].reset_index().rename(columns={'index':'index_med'}), on='_block_B')
    [['index_op','index_med']]
)

rr_B = 1 - len(pairs_B) / full_cross
print(f'Candidate pairs (B):  {len(pairs_B):>12,}')
print(f'Reduction ratio:      {rr_B*100:.6f}%')
op_tier2.drop(columns='_block_B', inplace=True)
med_clean.drop(columns='_block_B', inplace=True)
gc.collect()
print('✓ Strategy B complete.')

3.4 — STRATEGY B: EXACT LAST NAME + STATE
------------------------------------------------------------
Candidate pairs (B):       118,742
Reduction ratio:      99.997843%
✓ Strategy B complete.


## 3.5 — Strategy C: First Name Soundex + Last Name Soundex + State

Tightest 3-part key. The independence check in 3.2 confirms FN Soundex
and LN Soundex carry different information, so combining them adds
real discriminative power without redundancy.

In [5]:
print('3.5 — STRATEGY C: FN SOUNDEX + LN SOUNDEX + STATE')
print('-' * 60)

op_tier2['_block_C'] = (
    op_tier2['FIRST_NAME_SOUNDEX'].fillna('') + '_' +
    op_tier2['LAST_NAME_SOUNDEX'].fillna('') + '_' +
    op_tier2['Recipient_State'].fillna('')
)
med_clean['_block_C'] = (
    med_clean['FIRST_NAME_SOUNDEX'].fillna('') + '_' +
    med_clean['LAST_NAME_SOUNDEX'].fillna('') + '_' +
    med_clean['Rndrng_Prvdr_State_Abrvtn'].fillna('')
)

pairs_C = (
    op_tier2[['_block_C']].reset_index().rename(columns={'index':'index_op'})
    .merge(med_clean[['_block_C']].reset_index().rename(columns={'index':'index_med'}), on='_block_C')
    [['index_op','index_med']]
)

rr_C = 1 - len(pairs_C) / full_cross
print(f'Candidate pairs (C):  {len(pairs_C):>12,}')
print(f'Reduction ratio:      {rr_C*100:.6f}%')
op_tier2.drop(columns='_block_C', inplace=True)
med_clean.drop(columns='_block_C', inplace=True)
gc.collect()
print('✓ Strategy C complete.')

3.5 — STRATEGY C: FN SOUNDEX + LN SOUNDEX + STATE
------------------------------------------------------------
Candidate pairs (C):         2,795
Reduction ratio:      99.999949%
✓ Strategy C complete.


## 3.6 — Canopy Clustering Blocking

Canopy clustering is a **two-threshold** blocking method:
- Convert each record's `name + state` into a TF-IDF vector (character 3-grams)
- For each OP record, find all Medicare records within cosine distance **T2** (tight = 0.4)
- Records within **T1** (loose = 0.6) get removed from the candidate pool to avoid redundant centers

This catches fuzzy name matches that exact blocking misses, without comparing
every single pair. Processed per-state to keep memory manageable.

In [6]:
print('3.6 — CANOPY CLUSTERING BLOCKING')
print('-' * 60)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

T1 = 0.6
T2 = 0.4

def name_str(first, last, state):
    parts = [str(x).strip().upper() for x in [first, last, state] if pd.notna(x)]
    return ' '.join(p for p in parts if p and p != 'NAN')

op_tier2['_can'] = op_tier2.apply(lambda r: name_str(r['Covered_Recipient_First_Name'], r['Covered_Recipient_Last_Name'], r['Recipient_State']), axis=1)
med_clean['_can'] = med_clean.apply(lambda r: name_str(r['Rndrng_Prvdr_First_Name'], r['Rndrng_Prvdr_Last_Org_Name'], r['Rndrng_Prvdr_State_Abrvtn']), axis=1)

states_common = sorted(set(op_tier2['Recipient_State'].dropna().unique()) & set(med_clean['Rndrng_Prvdr_State_Abrvtn'].dropna().unique()))
canopy_list = []

for si, state in enumerate(states_common):
    op_st  = op_tier2[op_tier2['Recipient_State'] == state]
    med_st = med_clean[med_clean['Rndrng_Prvdr_State_Abrvtn'] == state]
    if len(op_st) == 0 or len(med_st) == 0: continue

    all_str = pd.concat([op_st['_can'].reset_index(drop=True), med_st['_can'].reset_index(drop=True)], ignore_index=True)
    n_op = len(op_st)

    tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3,3))
    try: mat = tfidf.fit_transform(all_str)
    except ValueError: continue

    dist = cosine_distances(mat[:n_op], mat[n_op:])
    op_idx, med_idx = op_st.index.tolist(), med_st.index.tolist()

    for i in range(n_op):
        for j in np.where(dist[i] <= T2)[0]:
            canopy_list.append((op_idx[i], med_idx[j]))

    if (si+1) % 10 == 0 or (si+1) == len(states_common):
        print(f'  {si+1}/{len(states_common)} states, pairs so far: {len(canopy_list):,}')

pairs_canopy = pd.DataFrame(canopy_list, columns=['index_op','index_med']).drop_duplicates()
rr_can = 1 - len(pairs_canopy) / full_cross
print(f'\nCanopy pairs:  {len(pairs_canopy):>12,}')
print(f'Reduction:     {rr_can*100:.6f}%')
print(f'Thresholds:    T1={T1}, T2={T2}')
op_tier2.drop(columns='_can', inplace=True)
med_clean.drop(columns='_can', inplace=True)
del canopy_list; gc.collect()
print('✓ Canopy complete.')

3.6 — CANOPY CLUSTERING BLOCKING
------------------------------------------------------------
  10/53 states, pairs so far: 8,321
  20/53 states, pairs so far: 9,505
  30/53 states, pairs so far: 11,174
  40/53 states, pairs so far: 14,238
  50/53 states, pairs so far: 18,738
  53/53 states, pairs so far: 18,815

Canopy pairs:        18,815
Reduction:     99.999658%
Thresholds:    T1=0.6, T2=0.4
✓ Canopy complete.


## 3.7 — LSH (Locality-Sensitive Hashing) Blocking

LSH is a probabilistic method that finds **approximately similar** records
without comparing every pair:

1. Break each `name + state` string into small character chunks (3-grams)
2. Create a compact fingerprint (MinHash signature) for each record
3. Hash fingerprints into buckets — similar records land in the same bucket

The similarity threshold is ~0.5 Jaccard. Records sharing roughly half
their character 3-grams become candidates. This catches spelling variations
that exact blocking misses entirely.

In [7]:
try:
    import datasketch; print('datasketch ready')
except ImportError:
    !pip install datasketch -q
    import datasketch; print('datasketch installed')

datasketch ready


In [8]:
print('3.7 — LSH BLOCKING')
print('-' * 60)

from datasketch import MinHash, MinHashLSH

NUM_PERM, LSH_THRESH, Q = 128, 0.5, 3

def mk_minhash(text):
    m = MinHash(num_perm=NUM_PERM)
    if not isinstance(text, str) or len(text) < Q: return None
    for i in range(len(text)-Q+1): m.update(text[i:i+Q].encode('utf-8'))
    return m

def ns(first, last, state):
    parts = [str(x).strip().upper() for x in [first,last,state] if pd.notna(x)]
    return ' '.join(p for p in parts if p and p != 'NAN')

op_tier2['_lsh'] = op_tier2.apply(lambda r: ns(r['Covered_Recipient_First_Name'], r['Covered_Recipient_Last_Name'], r['Recipient_State']), axis=1)
med_clean['_lsh'] = med_clean.apply(lambda r: ns(r['Rndrng_Prvdr_First_Name'], r['Rndrng_Prvdr_Last_Org_Name'], r['Rndrng_Prvdr_State_Abrvtn']), axis=1)

states_common = sorted(set(op_tier2['Recipient_State'].dropna().unique()) & set(med_clean['Rndrng_Prvdr_State_Abrvtn'].dropna().unique()))
print(f'States in common: {len(states_common)}')
lsh_list = []

for si, state in enumerate(states_common):
    op_st  = op_tier2[op_tier2['Recipient_State'] == state]
    med_st = med_clean[med_clean['Rndrng_Prvdr_State_Abrvtn'] == state]
    if len(op_st)==0 or len(med_st)==0: continue

    lsh = MinHashLSH(threshold=LSH_THRESH, num_perm=NUM_PERM)
    for idx, row in med_st.iterrows():
        mh = mk_minhash(row['_lsh'])
        if mh:
            try: lsh.insert(f'm_{idx}', mh)
            except ValueError: pass

    for idx_op, row_op in op_st.iterrows():
        mh_op = mk_minhash(row_op['_lsh'])
        if mh_op is None: continue
        for key in lsh.query(mh_op):
            lsh_list.append((idx_op, int(key.split('_')[1])))

    if (si+1) % 10 == 0 or (si+1) == len(states_common):
        print(f'  {si+1}/{len(states_common)} states, pairs so far: {len(lsh_list):,}')
    del lsh; gc.collect()

pairs_LSH = pd.DataFrame(lsh_list, columns=['index_op','index_med']).drop_duplicates()
rr_LSH = 1 - len(pairs_LSH) / full_cross
print(f'\nLSH pairs:     {len(pairs_LSH):>12,}')
print(f'Reduction:     {rr_LSH*100:.6f}%')
op_tier2.drop(columns='_lsh', inplace=True)
med_clean.drop(columns='_lsh', inplace=True)
del lsh_list; gc.collect()
print('✓ LSH complete.')

3.7 — LSH BLOCKING
------------------------------------------------------------
States in common: 53
  10/53 states, pairs so far: 45,651
  20/53 states, pairs so far: 52,368
  30/53 states, pairs so far: 61,172
  40/53 states, pairs so far: 82,225
  50/53 states, pairs so far: 99,900
  53/53 states, pairs so far: 100,196

LSH pairs:          100,196
Reduction:     99.998180%
✓ LSH complete.


## 3.8 — Union All Strategies & Deduplicate

In [9]:
print('3.8 — UNION & DEDUPLICATE')
print('-' * 60)

set_A      = set(map(tuple, pairs_A[['index_op','index_med']].values))
set_B      = set(map(tuple, pairs_B[['index_op','index_med']].values))
set_C      = set(map(tuple, pairs_C[['index_op','index_med']].values))
set_canopy = set(map(tuple, pairs_canopy[['index_op','index_med']].values))
set_LSH    = set(map(tuple, pairs_LSH[['index_op','index_med']].values))
set_all    = set_A | set_B | set_C | set_canopy | set_LSH

all_pairs = pd.DataFrame(list(set_all), columns=['index_op','index_med'])
print(f'Union total:  {len(all_pairs):>12,} unique pairs')

for name, s in [('A',set_A),('B',set_B),('C',set_C),('Canopy',set_canopy),('LSH',set_LSH)]:
    u = s - (set_all - s)
    print(f'  {name:<10} {len(s):>10,} total | {len(u):>10,} unique to {name}')
print('✓ Union complete.')

3.8 — UNION & DEDUPLICATE
------------------------------------------------------------
Union total:       492,427 unique pairs
  A             427,752 total |    427,752 unique to A
  B             118,742 total |    118,742 unique to B
  C               2,795 total |      2,795 unique to C
  Canopy         18,815 total |     18,815 unique to Canopy
  LSH           100,196 total |    100,196 unique to LSH
✓ Union complete.


## 3.9 — Blocking Quality Metrics

For each strategy: pair count, reduction ratio, OP coverage (how many
OP records appear in at least one pair), and recall ceiling (% of
"findable" OP records — those with an exact name+state match in Medicare).

In [10]:
print('3.9 — QUALITY METRICS')
print('-' * 60)

med_keys = set(zip(
    med_clean['Rndrng_Prvdr_First_Name'].str.upper(),
    med_clean['Rndrng_Prvdr_Last_Org_Name'].str.upper(),
    med_clean['Rndrng_Prvdr_State_Abrvtn']
))
findable = {idx for idx, r in op_tier2.iterrows()
    if (str(r['Covered_Recipient_First_Name']).upper(),
        str(r['Covered_Recipient_Last_Name']).upper(),
        str(r['Recipient_State'])) in med_keys}

print(f'Findable OP records: {len(findable):,} / {len(op_tier2):,}\n')

strats = {'A':set_A, 'B':set_B, 'C':set_C, 'Canopy':set_canopy, 'LSH':set_LSH, 'Union':set_all}
rows = []
print(f"{'Strategy':<10} {'Pairs':>12} {'RR %':>14} {'OP Cov':>8} {'Recall %':>10}")
print('=' * 56)
for name, ps in strats.items():
    n = len(ps)
    rr = (1 - n/full_cross)*100
    op_in = {p[0] for p in ps}
    recall = len(findable & op_in)/len(findable)*100 if findable else 0
    print(f'{name:<10} {n:>12,} {rr:>13.6f}% {len(op_in):>8,} {recall:>9.2f}%')
    rows.append({'strategy':name,'pairs':n,'reduction_ratio':1-n/full_cross,'op_coverage':len(op_in),'recall_ceiling_pct':round(recall,2)})

summary_df = pd.DataFrame(rows)
print('\n✓ Metrics complete.')

3.9 — QUALITY METRICS
------------------------------------------------------------
Findable OP records: 381 / 4,683

Strategy          Pairs           RR %   OP Cov   Recall %
A               427,752     99.992228%    4,574    100.00%
B               118,742     99.997843%    3,033    100.00%
C                 2,795     99.999949%    1,206    100.00%
Canopy           18,815     99.999658%    2,562    100.00%
LSH             100,196     99.998180%    3,720    100.00%
Union           492,427     99.991053%    4,622    100.00%

✓ Metrics complete.


## 3.10 — Export

In [11]:
print('3.10 — EXPORT')
print('-' * 60)

all_pairs.to_parquet(os.path.join(OUTPUT_DIR, 'candidate_pairs.parquet'), index=False)
print(f'✓ candidate_pairs.parquet ({len(all_pairs):,} pairs)')

summary_df.to_csv(os.path.join(OUTPUT_DIR, 'blocking_summary.csv'), index=False)
print(f'✓ blocking_summary.csv ({len(summary_df)} rows)')

print(f'\nAll artifacts → {OUTPUT_DIR}/')
for f in sorted(os.listdir(OUTPUT_DIR)):
    sz = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / 1024 / 1024
    print(f'  {f:<35s} {sz:>8.2f} MB')

print('\n' + '=' * 60)
print('PHASE 3 COMPLETE')
print('=' * 60)

3.10 — EXPORT
------------------------------------------------------------
✓ candidate_pairs.parquet (492,427 pairs)
✓ blocking_summary.csv (6 rows)

All artifacts → ../artifacts/phase3_blocking/
  blocking_summary.csv                    0.00 MB
  candidate_pairs.parquet                 3.16 MB

PHASE 3 COMPLETE
